## 1. Data Preparation

In [31]:
# read file
# import os

# file_names = os.listdir('corpus')
# file_content = ''

# for file_name in file_names:
#     file_path = os.path.join('corpus', file_name)

#     with open(file_path, 'r') as infile:
#         file_content = file_content + infile.read().replace('\n', '')

In [1]:
with open('corpus/corpus.txt', 'r') as infile:
        file_content = infile.read().replace('\n', '')

In [2]:
from nltk import sent_tokenize
sentences = sent_tokenize(file_content)

In [3]:
# splitting
training_size = int(len(sentences) * 0.7)
validation_size = int(len(sentences) * 0.1)

training_data = sentences[:training_size]
validation_data = sentences[training_size:training_size + validation_size]
test_data = sentences[training_size + validation_size:]

In [4]:
import re
from nltk import word_tokenize, ngrams
import string

emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)

number_backet_pattern = re.compile(r"\d")
                                
def tokenize_sentences(sentences):
    # convert to lower case
    low_sentences = [sentence.lower() for sentence in sentences]

    # remove number bracket
    cleaned_sentences = [re.sub(number_backet_pattern, "", sentence) for sentence in low_sentences]


    # remove emoji
    no_emoji_sentences = [re.sub(emoji_pattern, "", sentence) for sentence in cleaned_sentences]

    # split each sentence into tokens
    tokens_2d = [word_tokenize(sentence) for sentence in no_emoji_sentences]

    # remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    no_punc_tokens = []
    for tokens_1d in tokens_2d:
        no_punc_tokens.append([token.translate(translator) for token in tokens_1d])

    # # remove empty string
    no_empty_tokens = []
    for token_1d in no_punc_tokens:
        no_empty_tokens.append([token for token in token_1d if token != ''])

    # generate n-grams
    uni_grams = []
    bi_grams = []
    tri_grams = []
    four_grams = []

    for token_1d in no_empty_tokens:
        uni_grams.append(list(ngrams(token_1d, n=1)))
        bi_grams.append(list(ngrams(token_1d, n=2)))
        tri_grams.append(list(ngrams(token_1d, n=3)))
        four_grams.append(list(ngrams(token_1d, n=4)))

    # return a dictionary containing all lists of n-grams
    return {
        'sentences': no_empty_tokens,
        'uni_grams': uni_grams,
        'bi_grams': bi_grams,
        'tri_grams': tri_grams,
        'four_grams': four_grams,
    }

In [36]:
# handle unknown word

In [5]:
tokenized_sent = tokenize_sentences(training_data)

In [6]:
if (__name__ == '__main__'):
    for l in tokenized_sent['four_grams']:
        print(l)

[('computer', 'science', 'is', 'the'), ('science', 'is', 'the', 'study'), ('is', 'the', 'study', 'of'), ('the', 'study', 'of', 'computation'), ('study', 'of', 'computation', 'information'), ('of', 'computation', 'information', 'and'), ('computation', 'information', 'and', 'automation')]
[('computer', 'science', 'spans', 'theoretical'), ('science', 'spans', 'theoretical', 'disciplines'), ('spans', 'theoretical', 'disciplines', 'such'), ('theoretical', 'disciplines', 'such', 'as'), ('disciplines', 'such', 'as', 'algorithms'), ('such', 'as', 'algorithms', 'theory'), ('as', 'algorithms', 'theory', 'of'), ('algorithms', 'theory', 'of', 'computation'), ('theory', 'of', 'computation', 'and'), ('of', 'computation', 'and', 'information'), ('computation', 'and', 'information', 'theory'), ('and', 'information', 'theory', 'to'), ('information', 'theory', 'to', 'applied'), ('theory', 'to', 'applied', 'disciplines'), ('to', 'applied', 'disciplines', 'including'), ('applied', 'disciplines', 'includin

In [39]:
# from nltk.probability import FreqDist


# freq_uni = FreqDist()
# freq_bi = FreqDist()
# freq_tri = FreqDist()
# freq_four = FreqDist()

# # count uni-grams
# for l in tokenized_sent['uni_grams']:
#     for uni_gram in l:
#         freq_uni[uni_gram] = freq_uni[uni_gram] + 1

# # count bi-grams
# for l in tokenized_sent['bi_grams']:
#     for bi_gram in l:
#         freq_bi[bi_gram] = freq_bi[bi_gram] + 1
        
# # count bi-grams
# for l in tokenized_sent['tri_grams']:
#     for tri_gram in l:
#         freq_tri[tri_gram] = freq_tri[tri_gram] + 1

# # count bi-grams
# for l in tokenized_sent['four_grams']:
#     for four_gram in l:
#         freq_four[four_gram] = freq_four[four_gram] + 1


In [40]:
# if (__name__ == '__main__'):
#     for word in freq_four:
#         print(f'{word}: {freq_four[word]}')

In [7]:
def tokenize_words(data):
    words = []
    sentences = tokenize_sentences(data)['sentences']
    for sentence in sentences:
        words.append('<s>')
        words.extend(sentence)
        words.append('</s>')

    # create n-grams
    uni_grams = []
    bi_grams = []
    tri_grams = []
    four_grams = []

    uni_grams.extend(list(ngrams(words, n=1)))
    bi_grams.extend(list(ngrams(words, n=2)))
    tri_grams.extend(list(ngrams(words, n=3)))
    four_grams.extend(list(ngrams(words, n=4)))

    return {
        'sentences': words,
        'uni_grams': uni_grams,
        'bi_grams': bi_grams,
        'tri_grams': tri_grams,
        'four_grams': four_grams,
    } 
    

In [8]:
if (__name__ == '__main__'):
    for l in tokenize_words(training_data)['four_grams']:
        print(l)

('<s>', 'computer', 'science', 'is')
('computer', 'science', 'is', 'the')
('science', 'is', 'the', 'study')
('is', 'the', 'study', 'of')
('the', 'study', 'of', 'computation')
('study', 'of', 'computation', 'information')
('of', 'computation', 'information', 'and')
('computation', 'information', 'and', 'automation')
('information', 'and', 'automation', '</s>')
('and', 'automation', '</s>', '<s>')
('automation', '</s>', '<s>', 'computer')
('</s>', '<s>', 'computer', 'science')
('<s>', 'computer', 'science', 'spans')
('computer', 'science', 'spans', 'theoretical')
('science', 'spans', 'theoretical', 'disciplines')
('spans', 'theoretical', 'disciplines', 'such')
('theoretical', 'disciplines', 'such', 'as')
('disciplines', 'such', 'as', 'algorithms')
('such', 'as', 'algorithms', 'theory')
('as', 'algorithms', 'theory', 'of')
('algorithms', 'theory', 'of', 'computation')
('theory', 'of', 'computation', 'and')
('of', 'computation', 'and', 'information')
('computation', 'and', 'information', '

In [43]:
# tokenize_words()['uni_grams']
    

In [44]:
from nltk.probability import FreqDist


freq_uni = FreqDist()
freq_bi = FreqDist()
freq_tri = FreqDist()
freq_four = FreqDist()

tokenized_words = tokenize_words(training_data);

# count uni-grams
for uni_gram in tokenized_words['uni_grams']:
    freq_uni[uni_gram] = freq_uni[uni_gram] + 1

# count bi-grams
for bi_gram in tokenized_words['bi_grams']:
    freq_bi[bi_gram] = freq_bi[bi_gram] + 1
        
# count bi-grams
for tri_gram in tokenized_words['tri_grams']:
    freq_tri[tri_gram] = freq_tri[tri_gram] + 1

# count bi-grams
for four_gram in tokenized_words['four_grams']:
    freq_four[four_gram] = freq_four[four_gram] + 1


In [45]:
if (__name__ == '__main__'):
    for word in freq_four:
        print(f'{word}: {freq_four[word]}')

('aims', 'to', 'understand', 'and'): 2
('to', 'understand', 'and', 'process'): 2
('have', 'existed', 'since', 'antiquity'): 2
('<s>', 'computer', 'science', 'is'): 1
('computer', 'science', 'is', 'the'): 1
('science', 'is', 'the', 'study'): 1
('is', 'the', 'study', 'of'): 1
('the', 'study', 'of', 'computation'): 1
('study', 'of', 'computation', 'information'): 1
('of', 'computation', 'information', 'and'): 1
('computation', 'information', 'and', 'automation'): 1
('information', 'and', 'automation', '</s>'): 1
('and', 'automation', '</s>', '<s>'): 1
('automation', '</s>', '<s>', 'computer'): 1
('</s>', '<s>', 'computer', 'science'): 1
('<s>', 'computer', 'science', 'spans'): 1
('computer', 'science', 'spans', 'theoretical'): 1
('science', 'spans', 'theoretical', 'disciplines'): 1
('spans', 'theoretical', 'disciplines', 'such'): 1
('theoretical', 'disciplines', 'such', 'as'): 1
('disciplines', 'such', 'as', 'algorithms'): 1
('such', 'as', 'algorithms', 'theory'): 1
('as', 'algorithms', '